In [12]:
import time
import math
from sympy import isprime
import gmpy2
from gmpy2 import mpz, is_divisible

#Constantes
RATE = 8_995_888          # tests/s mesuré
MAX_DIRECT = 150_000_000  # impairs: ~17s
MAX_WIDEN  = 600_000_000  # impairs: ~67s
# =====================================================
# v4.3 (quantile + balayage + skip saturation + auto-widen)
#
# Objectif:
# 1) Balayage préalable sur k_est±1 + diagnostic
# 2) Ignorer les canaux saturés (peu informatifs) quand sqrt_flag != IN
# 3) Choisir un "meilleur" canal informatif (priorité sqrt_flag==IN)
# 4) AUTO-WIDEN: si échec, élargir keep_frac progressivement jusqu'au succès
#    (ou jusqu'à 1.00 = palier entier)
#
# Convention palier:
#   k => facteurs dans [2^k, 2^(k+1))
#
# Sonar:
#   |p_mid*q_mid - n| <= tau * n
# Sélection bins actifs:
#   quantile top keep_frac (ex 0.20 = top 20%)
# =====================================================

def bench_divisibility(n, start, count=2_000_000):
    # n = mpz(n)
    cand = start if (start & 1) else start + 1
    t0 = time.perf_counter()
    hits = 0
    for _ in range(count):
        if is_divisible(n, cand):
            hits += 1
        cand += 2
    dt = time.perf_counter() - t0
    rate = count / dt
    print(f"Bench: {count:,} tests in {dt:.3f}s → {rate:,.0f} tests/s (hits={hits})")
    return rate
    
def ask_n_only() -> int:
    s = input("Entrez n (décimal, ou 0x... en hex) : ").strip()
    if not s:
        raise ValueError("Entrée vide.")
    n = int(s, 16) if s.lower().startswith("0x") else int(s)
    if n <= 3:
        raise ValueError("n doit être > 3")
    if n % 2 == 0:
        raise ValueError("n est pair : cas trivial (facteur 2). Donne un n impair.")
    return n

def estimate_k_from_n(n: int) -> int:
    return max(2, math.isqrt(n).bit_length() - 1)

def band_bounds(k: int):
    return (1 << k), (1 << (k + 1))

def _quantile_threshold(values, keep_frac: float):
    keep_frac = max(0.0, min(1.0, keep_frac))
    if keep_frac <= 0:
        return float("inf")
    if keep_frac >= 1:
        return float("-inf")
    vs = sorted(values)
    q = 1.0 - keep_frac
    idx = int(q * (len(vs) - 1))
    return vs[idx]

def refined_channel_quantile(
    n: int,
    k: int,
    B: int = 128,
    tau: float = 0.10,
    keep_frac: float = 0.20,
):
    low, high_excl = band_bounds(k)
    high = high_excl - 1
    width = high - low + 1

    # Prior PNT (densité décroissante)
    density = [1.0 / math.log(low + (i + 0.5) * width / B + 2.0) for i in range(B)]
    total = sum(density) or 1.0
    density = [d / total for d in density]

    tol = tau * n
    weights = [0.0] * B
    step = width / B

    for i in range(B):
        p_mid = low + (i + 0.5) * step
        for j in range(B):
            q_mid = low + (j + 0.5) * step
            if abs(p_mid * q_mid - n) <= tol:
                w = density[i] * density[j]
                weights[i] += w
                weights[j] += w

    max_w = max(weights) if max(weights) > 0 else 1.0
    min_w = min(weights)
    flat_ratio = (min_w / max_w) if max_w > 0 else 0.0

    keep_frac = max(0.01, min(1.0, float(keep_frac)))
    threshold = _quantile_threshold(weights, keep_frac)
    active_bins = [i for i, w in enumerate(weights) if w >= threshold]

    if not active_bins:
        diag = {
            "B": B,
            "tau": tau,
            "keep_frac": keep_frac,
            "min_w": min_w,
            "max_w": max_w,
            "flat_ratio": flat_ratio,
            "active_bins": 0,
            "threshold": threshold,
            "fallback": True,
        }
        return [(low, high)], 1.0, diag

    active_bins.sort()

    # Intervalles continus
    intervals = []
    start_idx = active_bins[0]
    for t in range(1, len(active_bins)):
        if active_bins[t] != active_bins[t - 1] + 1:
            start = int(low + start_idx * step)
            end = int(low + (active_bins[t - 1] + 1) * step)
            intervals.append((start, min(end, high)))
            start_idx = active_bins[t]
    start = int(low + start_idx * step)
    end = int(low + (active_bins[-1] + 1) * step)
    intervals.append((start, min(end, high)))

    covered = sum(e - s + 1 for s, e in intervals)
    coverage = covered / width

    diag = {
        "B": B,
        "tau": tau,
        "keep_frac": keep_frac,
        "min_w": min_w,
        "max_w": max_w,
        "flat_ratio": flat_ratio,
        "active_bins": len(active_bins),
        "threshold": threshold,
        "fallback": False,
    }
    return intervals, coverage, diag

def is_saturated(coverage: float, active_bins: int, B: int, fallback: bool, cov_sat: float = 0.95, bins_sat: float = 0.95) -> bool:
    if fallback:
        return True
    if coverage >= cov_sat:
        return True
    if active_bins / max(1, B) >= bins_sat:
        return True
    return False

def total_odd_candidates(intervals):
    tot = 0
    for low, high in intervals:
        a = low if (low & 1) else low + 1
        if a <= high:
            tot += ((high - a) // 2) + 1
    return tot


def chalut_avec_progress(n, intervals, progress_step=10_000_000):
    sqrt_n = int(gmpy2.isqrt(n)) + 1
    tested = 0

    for low, high in intervals:
        if low > sqrt_n:
            print(f"Exploration partie haute [{low:,} – {high:,}]...")
            start = low if low % 2 == 1 else low + 1
            cand = start
            while cand <= high:
                tested += 1
                if tested % progress_step == 0:
                    print(f"  → {tested // 1_000_000} millions de candidats testés")
                if is_divisible(n, cand) and isprime(cand):
                    small = n // cand
                    if isprime(small):
                        print(f"\nFACTEUR TROUVÉ : {cand} (grand)")
                        return cand, small
                cand += 2
        else:
            effective_high = min(high, sqrt_n)
            if low <= effective_high:
                start = low if low % 2 == 1 else low + 1
                cand = start
                while cand <= effective_high:
                    tested += 1
                    if tested % progress_step == 0:
                        print(f"  → {tested // 1_000_000} millions de candidats testés")
                    if is_divisible(n, cand) and isprime(cand):
                        large = n // cand
                        if isprime(large):
                            print(f"\nFACTEUR TROUVÉ : {cand} (petit)")
                            return cand, large
                    cand += 2
    return None, None

def pretty_span(intervals):
    if not intervals:
        return "(none)"
    lo = min(s for s, _ in intervals)
    hi = max(e for _, e in intervals)
    return f"[{lo:,} – {hi:,}]"

# ====================== EXÉCUTION ======================
print("=== Sonar dyadique + Chalut (n-only) — v4.3 auto-widen ===\n")

# 0) Lire n (int pour le sonar) + version mpz pour le chalut
n_int = ask_n_only()
n = mpz(n_int)

sqrt_n = math.isqrt(n_int)  # une seule fois (int)
print(f"\nn = {n_int} (bitlen={n_int.bit_length()})")
print(f"sqrt(n) = {sqrt_n} (bitlen={sqrt_n.bit_length()})\n")

k_est = estimate_k_from_n(n_int)
print(f"Estimation automatique du palier k : {k_est}\n")

print("=== Paramètres sonar (quantile) ===\n")
print("tau : tolérance relative sur |p*q - n| (ex: 0.05 = ±5% autour de n)")
print("keep_frac : fraction top X% de bins conservés (ex: 0.20 = top 20%)\n")

tau_user = float(input("Entrez tau (recommandé 0.03–0.10) : ") or 0.05)
keep_user = float(input("Entrez keep_frac (quantile, ex 0.20) : ") or 0.20)

# Auto-widen schedule (on inclut keep_user et on monte progressivement)
schedule = [keep_user, 0.30, 0.45, 0.60, 0.80, 1.00]
schedule = sorted(set(max(0.01, min(1.0, float(x))) for x in schedule))

# 1) BALAYAGE préalable sur k_est±1 (avec keep_user)
k_candidates = sorted({max(2, k_est - 1), k_est, k_est + 1})
scan = []

print("\n--- Balayage préalable (k_est±1) ---")
print("(band = [2^k, 2^(k+1)) ; sqrt-flag indique si sqrt(n) tombe dans ce palier)\n")

for k_try in k_candidates:
    intervals, coverage, diag = refined_channel_quantile(
        n_int, k_try, B=256, tau=tau_user, keep_frac=keep_user
    )
    lo, hi = band_bounds(k_try)
    sqrt_flag = "IN" if (lo <= sqrt_n < hi) else "--"
    sat = is_saturated(coverage, diag["active_bins"], diag["B"], diag["fallback"])

    row = {
        "k": k_try,
        "lo": lo,
        "hi": hi,
        "sqrt_flag": sqrt_flag,
        "intervals": intervals,
        "coverage": coverage,
        "diag": diag,
        "saturated": sat,
    }
    scan.append(row)

    print(f"k={k_try:<3} | band=[{lo:,} – {hi:,}) | sqrt:{sqrt_flag} | coverage={coverage:6.2%} | #int={len(intervals):<3} | span={pretty_span(intervals)}")
    print(f"      diag: active_bins={diag['active_bins']}/{diag['B']} | flat(min/max)={diag['flat_ratio']:.3f} | fallback={diag['fallback']} | saturated={sat}")

print("-------------------------------\n")

# 2) Choix du meilleur palier à chaluter
def rank_row(r):
    return (
        0 if r["sqrt_flag"] == "IN" else 1,
        0 if not r["saturated"] else 1,
        r["coverage"],
        abs(r["k"] - k_est),
    )

scan_sorted = sorted(scan, key=rank_row)
primary = scan_sorted[0]
k_primary = primary["k"]

print(f"Palier prioritaire : k={k_primary} (sqrt:{primary['sqrt_flag']}, saturated={primary['saturated']}, coverage={primary['coverage']:.2%})\n")

# 3) Chalut sur palier prioritaire avec AUTO-WIDEN
print("--- Chalut (auto-widen sur le palier prioritaire) ---")
p_found = q_found = None
k_success = None
keep_success = None

for keep in schedule:
    intervals, coverage, diag = refined_channel_quantile(
        n_int, k_primary, B=128, tau=tau_user, keep_frac=keep
    )

    print(f"[widen] k={k_primary} | keep_frac={keep:.2f} | "
          f"coverage={coverage*100:.2f}% | #int={len(intervals)} | "
          f"span={pretty_span(intervals)}")

    Nodd = total_odd_candidates(intervals)
    Tsec = Nodd / RATE
    print(f"[epaisseur] odd={Nodd:,}  est_time={Tsec:.1f}s  ({Tsec/60:.1f} min)")

    if Nodd > MAX_WIDEN:
        print("→ Trop large : refine requis (ou keep_frac plus bas).")
        continue

    p_try, q_try = chalut_avec_progress(n, intervals)

    if p_try is not None:
        p_found, q_found = p_try, q_try
        k_success = k_primary
        keep_success = keep
        print(f"\n✓ Succès avec k={k_primary} et keep_frac={keep:.2f}")
        break

# 4) Plan B si échec
if p_found is None:
    print("--- Plan B : autres paliers (si disponibles) ---")
    alts = [r for r in scan if r["k"] != k_primary]
    alts.sort(key=rank_row)
    for r in alts:
        if r["saturated"] and r["sqrt_flag"] != "IN":
            print(f"skip k={r['k']} (saturé & sqrt!=IN)")
            continue
        k_try = r["k"]
        print(f"→ Tentative brute sur k={k_try} (sans auto-widen) | sqrt:{r['sqrt_flag']} | saturated={r['saturated']} | coverage={r['coverage']:.2%}")
        intervals = r["intervals"]
        p_try, q_try = chalut_avec_progress(n, intervals)
        if p_try is not None:
            p_found, q_found = p_try, q_try
            k_success = k_try
            keep_success = keep_user
            print(f"\n✓ Succès avec k={k_try}\n")
            break
        else:
            print(f"✗ échec sur k={k_try}\n")

# 5) Résultat
if p_found is not None:
    assert p_found * q_found == n
    assert n % p_found == 0 and n % q_found == 0
    print(f"FACTEURS TROUVÉS : {p_found} × {q_found} = {p_found * q_found}")
    if keep_success is not None:
        print(f"✓ Succès – factorisation complète (k={k_success}, keep_frac={keep_success:.2f}, tau={tau_user})")
    else:
        print(f"✓ Succès – factorisation complète (k={k_success}, tau={tau_user})")
else:
    print("Aucun facteur trouvé")


=== Sonar dyadique + Chalut (n-only) — v4.3 auto-widen ===



Entrez n (décimal, ou 0x... en hex) :  2309503435168192423



n = 2309503435168192423 (bitlen=62)
sqrt(n) = 1519705048 (bitlen=31)

Estimation automatique du palier k : 30

=== Paramètres sonar (quantile) ===

tau : tolérance relative sur |p*q - n| (ex: 0.05 = ±5% autour de n)
keep_frac : fraction top X% de bins conservés (ex: 0.20 = top 20%)



Entrez tau (recommandé 0.03–0.10) :  0.04
Entrez keep_frac (quantile, ex 0.20) :  0.20



--- Balayage préalable (k_est±1) ---
(band = [2^k, 2^(k+1)) ; sqrt-flag indique si sqrt(n) tombe dans ce palier)

k=29  | band=[536,870,912 – 1,073,741,824) | sqrt:-- | coverage=100.00% | #int=1   | span=[536,870,912 – 1,073,741,823]
      diag: active_bins=256/256 | flat(min/max)=0.000 | fallback=False | saturated=True
k=30  | band=[1,073,741,824 – 2,147,483,648) | sqrt:IN | coverage=20.31% | #int=4   | span=[1,103,101,952 – 1,342,177,280]
      diag: active_bins=52/256 | flat(min/max)=0.282 | fallback=False | saturated=False
k=31  | band=[2,147,483,648 – 4,294,967,296) | sqrt:-- | coverage=100.00% | #int=1   | span=[2,147,483,648 – 4,294,967,295]
      diag: active_bins=256/256 | flat(min/max)=0.000 | fallback=False | saturated=True
-------------------------------

Palier prioritaire : k=30 (sqrt:IN, saturated=False, coverage=20.31%)

--- Chalut (auto-widen sur le palier prioritaire) ---
[widen] k=30 | keep_frac=0.20 | coverage=21.09% | #int=2 | span=[1,107,296,256 – 1,350,565,888]


In [8]:
rate = bench_divisibility(n, start=intervals[0][0], count=2_000_000)

Bench: 2,000,000 tests in 0.222s → 8,995,888 tests/s (hits=0)
